In [2]:
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import requests
import json

In [3]:
def print_response(response):

    print('response status:', json_response.status_code)
    print('response text: ', json_response.text)

In [4]:
base_url = 'http://127.0.0.1:8501/v1/models/'

In [5]:
# Check that model is available 
json_response = requests.get(base_url+'retrieval')
print_response(json_response)

response status: 200
response text:  {
 "model_version_status": [
  {
   "version": "3",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}



In [6]:
# Check that model is available 
json_response = requests.get(base_url + 'retrieval_exclusion')
print_response(json_response)

response status: 200
response text:  {
 "model_version_status": [
  {
   "version": "6",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}



In [7]:
# test model with an image query
img_path = '../data/raw/ethz/images/0/18.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img) / 255
x = preprocess_input(x) 
# x = np.expand_dims(x, axis=0)
qry_img = x.tolist()


In [9]:
headers = {"content-type": "application/json"}
data = json.dumps({"instances":[qry_img,]})
json_response = requests.post(base_url + 'retrieval:predict',
                              data=data,
                              headers=headers)
predictions = json.loads(json_response.text)

predictions

{'predictions': [{'output_1': [140.631317,
    139.639084,
    139.30835,
    138.536621,
    137.985382,
    137.875137,
    137.875137,
    137.213654,
    136.772675,
    135.670197,
    135.670197,
    135.339462,
    135.188324,
    135.118973,
    135.118973,
    135.008728,
    135.008728,
    134.567734,
    134.526855,
    134.457489,
    134.416595,
    134.347244,
    133.465271,
    133.424377,
    133.355011,
    133.244766,
    133.093643,
    133.093643,
    133.024277,
    132.983398,
    132.873138,
    132.693542,
    132.583282,
    132.542404,
    132.473038,
    132.142303,
    132.069885,
    132.032059,
    132.007095,
    131.921814,
    131.896851,
    131.770676,
    131.701309,
    131.660431,
    131.550186,
    131.550186,
    131.48082,
    131.408401,
    131.370575,
    131.125122,
    131.125122,
    131.039841,
    131.039841,
    130.929581,
    130.888702,
    130.888702,
    130.857162,
    130.778458,
    130.668213,
    130.668213,
    130.598846,

In [20]:
qry_meta = np.zeros(126)
qry_meta[-10]=1

qry_meta = qry_meta.tolist()
qry_years = np.array([0,0]).tolist()

headers = {"content-type": "application/json"}
data = {"instances": [
            {        
            "input_1":qry_img,
            "input_2":qry_meta,
            "input_3":qry_years
            },
        ]
       }

data = json.dumps(data)
json_response = requests.post(base_url + 'retrieval_exclusion:predict',
                              data=data,
                              headers=headers)
predictions = json.loads(json_response.text)

predictions

{'predictions': [{'output_1': [0.91360873,
    0.877594411,
    0.874944568,
    0.874944568,
    0.863157511,
    0.862585723,
    0.85544467,
    0.850550592,
    0.848533034,
    0.846106768,
    0.845652699,
    0.845527112,
    0.843419909,
    0.841391563,
    0.83863616,
    0.838249564,
    0.837044239,
    0.835544944,
    0.835425198,
    0.835261703,
    0.83370024,
    0.831737936,
    0.827821195,
    0.82433182,
    0.823963106,
    0.823021412,
    0.821438491,
    0.821246922,
    0.820257783,
    0.818709254,
    0.818709254,
    0.818429589,
    0.813013613,
    0.810701191,
    0.809994757,
    0.80947,
    0.808218598,
    0.806598425,
    0.806548417,
    0.806185484,
    0.804693103,
    0.801254451,
    0.800471187,
    0.80038774,
    0.80009377,
    0.798300564,
    0.797796249,
    0.79645741,
    0.795955896,
    0.795645475,
    0.795626521,
    0.795405149,
    0.794822156,
    0.794520915,
    0.794104874,
    0.791988671,
    0.790413439,
    0.790266097,

In [118]:
records_list = predictions["predictions"][0]["output_2"]
records_list

[10570, 196511, 32182, 34250, 11428, 203490, 8563, 11495, 14856, 247925]

In [129]:
data = {
        "instances": [{
            "queries": x,
            "exclusions":records_list,
            }]
    }
data = {"signature_name": "query_with_exclusions", 
        "instances": [
                {'queries':x, 'exclusions':records_list}
            ]
       }
data = json.dumps(data)

headers = {"content-type": "application/json"}
json_response = requests.post('http://127.0.0.1:8501/v1/models/model/versions/1:predict',
                              data=data,
                              headers=headers)
print_response(json_response)
predictions = json.loads(json_response.text)
predictions

response status: 400
response text:  {
    "error": "Serving signature name: \"query_with_exclusions\" not found in signature def"
}


{'error': 'Serving signature name: "query_with_exclusions" not found in signature def'}

In [37]:
# check if the response is ok


In [ ]:
headers = {"content-type": "application/json"}
data = json.dumps({"instances":x})
json_response = requests.post('http://127.0.0.1:8501/v1/models/model/versions/1:predict',
                              data=data,
                              headers=headers)

In [61]:
MODEL_NAME="model"
VERSION=1
metadata_url = f"""http://localhost:8501/v1/models/{MODEL_NAME}/versions/{VERSION}/metadata"""
metadata_url

'http://localhost:8501/v1/models/model/versions/1/metadata'

In [30]:
resp = requests.get(metadata_url)
print(resp.text)

{
"model_spec":{
 "name": "model",
 "signature_name": "",
 "version": "1"
}
,
"metadata": {"signature_def": {
 "signature_def": {
  "serving_default": {
   "inputs": {
    "input_1": {
     "dtype": "DT_FLOAT",
     "tensor_shape": {
      "dim": [
       {
        "size": "-1",
        "name": ""
       },
       {
        "size": "224",
        "name": ""
       },
       {
        "size": "224",
        "name": ""
       },
       {
        "size": "3",
        "name": ""
       }
      ],
      "unknown_rank": false
     },
     "name": "serving_default_input_1:0"
    }
   },
   "outputs": {
    "output_1": {
     "dtype": "DT_FLOAT",
     "tensor_shape": {
      "dim": [],
      "unknown_rank": true
     },
     "name": "StatefulPartitionedCall:0"
    },
    "output_2": {
     "dtype": "DT_INT32",
     "tensor_shape": {
      "dim": [],
      "unknown_rank": true
     },
     "name": "StatefulPartitionedCall:1"
    }
   },
   "method_name": "tensorflow/serving/predict"
  },
  "__s

In [22]:
predictions = json.loads(json_response.text)
predictions

{'predictions': [[-0.587940514,
   -0.587940514,
   1.08393431,
   -0.587940514,
   -0.343698233,
   -0.587940514,
   -0.587940514,
   1.95080197,
   -0.587940514,
   -0.587940514,
   -0.587940514,
   0.623739481,
   1.4989984,
   -0.587940514,
   0.425772786,
   0.76575309,
   -0.587940514,
   0.7428087,
   -0.587940514,
   -0.587940514,
   -0.570832968,
   -0.587940514,
   0.715191901,
   -0.587940514,
   -0.587940514,
   -0.587940514,
   -0.587940514,
   -0.554082036,
   2.20294881,
   -0.587940514,
   0.596960604,
   -0.587940514,
   1.99113894,
   -0.587940514,
   1.02717376,
   -0.587940514,
   -0.381201833,
   -0.404672563,
   1.27380168,
   -0.587940514,
   -0.155041203,
   -0.587940514,
   0.246595442,
   -0.360376984,
   -0.38137871,
   -0.0356519073,
   0.123194896,
   -0.587940514,
   0.00391065236,
   -0.543495536,
   0.243136451,
   1.77016056,
   2.50888658,
   -0.587940514,
   -0.587940514,
   0.412113488,
   0.0421610922,
   1.0635457,
   -0.338707358,
   -0.587940514,